In [1]:
import findspark
findspark.init("C:\spark")

In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.3.1 pyspark-shell'

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [5]:
spark = SparkSession.builder \
.master("local[4]") \
.appName("ReadFromKafka") \
.getOrCreate()

In [6]:
df = spark \
.readStream \
.format("kafka") \
.option("kafka.bootstrap.servers","localhost:9092") \
.option("subscribe","deneme") \
.load()

In [7]:
df2 = df.selectExpr(" CAST(key AS STRING)", "CAST(value AS STRING)")

In [8]:
words = df2.select(explode(split(col("value"), " ")).alias("value"))


In [9]:
word_conuts = words.groupBy("value").count().sort(desc("count"))


In [10]:
query = word_conuts.writeStream \
.format("console") \
.outputMode("complete") \
.trigger(processingTime='1 seconds') \
.start()

In [11]:
query = word_conuts.writeStream
